# Pharmaceutical and Non-pharmaceutical interventions

April 13

In [63]:
import pandas as pd
import seaborn as sns
import matplotlib as plt 

In [64]:
#Read train 
deaths = pd.read_csv('data/deaths.csv')
cases = pd.read_csv('data/cases.csv')
deaths.drop(['Province/State'], axis=True)
deaths.shape

(274, 84)

In [65]:
df = pd.read_csv('Global_variable_per_country.csv')

In [66]:
df.shape

(274, 134)

### Test

https://www.kaggle.com/skylord/covid19-tests-conducted-by-country

In [67]:
test = pd.read_csv('variable/Intervention/test/Tests_conducted_09April2020.csv')

In [68]:
len(test['Country or region'].unique())

126

In [69]:
test.drop(['Ref.', 'Source_1','Source_2'], axis=1, inplace=True)

In [70]:
df = df.merge(test, how='left', left_on='Territory', right_on='Country or region')
df.drop(['Country or region'], axis=1, inplace=True)

In [71]:
df["Tests"] =  df["Tests"].fillna(0)
df.drop(['As of'], axis=1, inplace=True)



(274, 138)

In [102]:
df.shape

(274, 138)

247 counties without info relative to test !

Maybe find better data.

### Lockdown 

https://www.kaggle.com/jcyzag/covid19-lockdown-dates-by-country

In [99]:
lock = pd.read_csv('variable/Intervention/lockdown/countryLockdowndates.csv')
lock.drop(['Province', 'Reference'], axis=1, inplace=True)
lock.rename(columns={'Country/Region':'reg','Date':'lockdown_date',
                        'Type':'Type_lockdown'}, inplace=True)
lock =lock.dropna()
lock = lock.drop_duplicates(subset=['reg'], keep='last')

In [100]:
lock.head(1)

,reg,lockdown_date,Type_lockdown
0,Afghanistan,24/03/2020,Full


In [105]:
df = df.merge(lock, how='left', left_on='Territory', right_on='reg')

df.drop(['Country/Region_y'], axis=1, inplace=True)

In [106]:
df.shape

(274, 141)

## Vaccine - clinical test
https://www.clinicaltrials.gov/ct2/results?cond=COVID-19

https://www.who.int/ictrp/en/

https://www.kaggle.com/zohrarezgui/coronavirus-clinical-trials-dataset

In [107]:
trial = pd.read_csv('variable/Intervention/COVID19-927-trials.csv')
trial = trial[['Countries', 'Date registration','Date enrollement','Target size']]
trial.shape

(927, 4)

We will take the sum of the number of clinical trials.

In [108]:
df.shape

(274, 141)

In [109]:
nb_clinical_trial = list(trial.groupby('Countries').size())
country = list(trial.sort_values('Countries').Countries.unique())[0:62]
trial2 = pd.DataFrame({'nb_trial':nb_clinical_trial ,'country' :country })

In [110]:
def tidy_split(df, column, sep=';', keep=False):
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [111]:
trial2 = tidy_split(trial2, 'country')

In [112]:
nb_clinical_trial = list(trial2.groupby('country').size())
country = list(trial2.sort_values('country').country.unique())
final_trial = pd.DataFrame({'nb_trial':nb_clinical_trial ,'country' :country })
final_trial.head(4)

,nb_trial,country
0,1,Argentina
1,9,Australia
2,1,Austria
3,5,Belgium


In [116]:
df.shape

(274, 143)

In [114]:
df = df.merge(final_trial, how='left', left_on='Territory', right_on='country')

In [115]:
df["nb_trial"] =  df["nb_trial"].fillna(0)

In [117]:
df.to_csv('Global_variable_per_country+Intervention.csv',index=False)